# Classificador de Risco Médico com TF-IDF e Regressão Logística

Este notebook demonstra a criação de um classificador de texto para identificar frases de diagnósticos médicos como de 'alto risco' ou 'baixo risco'. Utilizamos TF-IDF para vetorização do texto e Regressão Logística para a classificação.

## 1. Configuração e Importação de Bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

## 2. Carregamento dos Dados

Carregamos o arquivo `diagnosticos.csv` que contém as frases e suas respectivas situações de risco.

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/FIAP/FASE 1 2025/diagnosticos/diagnosticos.csv')
print(data.head())

                                        frase     situacao
0            sinto dor no peito e falta de ar   alto risco
1            tive um leve incômodo nas costas  baixo risco
2  meu coração está acelerado e sinto tontura   alto risco
3    sinto um cansaço leve depois de caminhar  baixo risco
4         perdi o apetite e estou com náuseas   alto risco


## 3. Preparação dos Dados

Dividimos os dados em conjuntos de treinamento e teste e aplicamos a vetorização TF-IDF.

In [ ]:
# Separar features (X) e target (y)
X = data['frase']
y = data['situacao']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar e treinar o vetorizador TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f'Shape de X_train_vec: {X_train_vec.shape}')
print(f'Shape de X_test_vec: {X_test_vec.shape}')

Shape de X_train_vec: (16, 64)
Shape de X_test_vec: (4, 64)


## 4. Treinamento do Modelo

Utilizamos um modelo de Regressão Logística para classificar as frases.

In [ ]:
model = LogisticRegression(max_iter=1000) # Aumentar max_iter para convergência
model.fit(X_train_vec, y_train)

print('Modelo treinado com sucesso.')

Modelo treinado com sucesso.


## 5. Avaliação do Modelo

Avaliamos o desempenho do modelo nos dados de teste.

In [ ]:
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia do modelo: {accuracy:.2f}')
print('Relatório de Classificação:')
print(classification_report(y_test, y_pred))

Acurácia do modelo: 0.75
Relatório de Classificação:
              precision    recall  f1-score   support

  alto risco       0.50      1.00      0.67         1
 baixo risco       1.00      0.67      0.80         3

    accuracy                           0.75         4
   macro avg       0.75      0.83      0.73         4
weighted avg       0.88      0.75      0.77         4



## 6. Salvamento do Modelo e Vetorizador

Salvamos o modelo treinado e o vetorizador TF-IDF para uso futuro.

In [ ]:
joblib.dump(model, 'risk_classifier_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print('Modelo e vetorizador salvos com sucesso.')

Modelo e vetorizador salvos com sucesso.


## 7. Função de Predição

Criamos uma função para prever o risco de novas frases usando o modelo salvo.

In [ ]:
def predict_risk(phrase):
    # Carregar o modelo e o vetorizador (se não estiverem já carregados)
    if not os.path.exists('risk_classifier_model.pkl') or not os.path.exists('tfidf_vectorizer.pkl'):
        print('Erro: Modelo ou vetorizador não encontrados. Certifique-se de que foram salvos.')
        return None

    loaded_model = joblib.load('risk_classifier_model.pkl')
    loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')

    # Vetorizar a frase de entrada
    phrase_vec = loaded_vectorizer.transform([phrase])
    # Fazer a predição
    prediction = loaded_model.predict(phrase_vec)
    return prediction[0]

# Testando a função de predição
print('Testando a função de predição:')
test_phrases = [
    'sinto uma dor muito forte no peito e estou suando frio',
    'estou com um pouco de tosse e dor de cabeça leve',
    'minha perna está dormente e não consigo mexer o braço',
    'senti um pequeno desconforto no estômago depois de comer',
    'febre alta e dificuldade para respirar'
]

for phrase in test_phrases:
    risk = predict_risk(phrase)
    print(f"Frase: '{phrase}' -> Risco: {risk}")

Testando a função de predição:
Frase: 'sinto uma dor muito forte no peito e estou suando frio' -> Risco: alto risco
Frase: 'estou com um pouco de tosse e dor de cabeça leve' -> Risco: baixo risco
Frase: 'minha perna está dormente e não consigo mexer o braço' -> Risco: alto risco
Frase: 'senti um pequeno desconforto no estômago depois de comer' -> Risco: baixo risco
Frase: 'febre alta e dificuldade para respirar' -> Risco: alto risco
